In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('corona.tsv', delimiter='\t', quoting=3)

In [15]:
import re
import nltk
import tkinter
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []

y = dataset.iloc[:, -1].values.tolist()
y.pop(-1)
removerindex = []

for i in range(len(dataset)):

  removeurl = dataset['OriginalTweet'][i]
  removeurl = removeurl.split()
  
  for k in range(len(removeurl)):
    removeurl[k] = re.sub(r'^https?:\/\/.*[\r\n]*', '', removeurl[k], flags=re.MULTILINE)
  removeurl = ' '.join(removeurl)

  removeusernames = filter(lambda x:x[0]!='@', removeurl.split())
  removeusernames = ' '.join(removeusernames)

  removehash = filter(lambda x:x[0]!='#', removeusernames.split())
  removehash = ' '.join(removehash)

  tweet = re.sub('[^a-zA-Z]', ' ', removehash)
  tweet = tweet.lower()
  tweet = tweet.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  tweet = [ps.stem(word) for word in tweet if not word in set(all_stopwords)]
  tweet = ' '.join(tweet)
  if tweet=='':
    removerindex.append(i)
  else:
    corpus.append(tweet)

for h in range(len(removerindex)-1):
  y.pop(removerindex[h])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\supre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=25000)
x = cv.fit_transform(corpus).toarray()

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [18]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)

GaussianNB()

In [19]:
y_pred = gnb.predict(x_test)

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2675  393]
 [4279  874]]


0.4316993066536918